## Imports

In [ ]:
import pandas as pd
import fnmatch
from lxml import etree
import os
import numpy as np
import xml.etree.ElementTree as ET
from tqdm import tqdm

In [ ]:
# DIRETÓRIO TABELA PRODUTOS
DIRETORIO_TABELA_PRODUTOS = "{}" 
 # DIRETÓRIO TABELA CLASSIFICAÇÃO
DIRETORIO_TABELA_CLASSIFICADOR = "{}"
# DIRETÓRIO DOS XMLS
DIRETORIO_ENTRADA_XML = "{}"  
# DIRETÓRIO SAÍDA DOS XMLS
DIRETORIO_SAIDA_XML = "{}"  
# DIRETÓRIO SAÍDA DOS ARQUIVOS QUE PRECISAM DE UMA SEGUNDA VERIFICAÇÃO
DIRETORIO_SAIDA_cProd_NAO_ENCONTRADOS = "{}" 
# DIRETÓRIO DE SAÍDA DOS XMLS VERIFICADOS PELA SEGUNDA VEZ
DIRETORIO_SAIDA_XML_SEGUNDA_VERIFICACAO = "{}"
# DIRETÓRIO DE SAÍDA DOS XMLS VERIFICADOS PELA SEGUNDA VEZ ONDE OS cPROD NÃO FORAM TROCADOS
DIRETORIO_SAIDA_XML_SEGUNDA_VERIFICACAO_cProd = "{}"

# CLASSIFICADOR DOS GRUPOS EXCEL
CLASSIFICADOR = pd.read_excel(DIRETORIO_TABELA_CLASSIFICADOR)
# BASE DO EXCEL DE PRODUTOS
PRODUTOS = pd.read_excel(DIRETORIO_TABELA_PRODUTOS)

# DIRETORIO_SAIDA_XML
if not os.path.exists(DIRETORIO_SAIDA_XML):
    os.makedirs(DIRETORIO_SAIDA_XML)

# DIRETORIO_SAIDA_cProd_NAO_ENCONTRADOS
if not os.path.exists(DIRETORIO_SAIDA_cProd_NAO_ENCONTRADOS):
    os.makedirs(DIRETORIO_SAIDA_cProd_NAO_ENCONTRADOS)

# DIRETORIO_SAIDA_XML_SEGUNDA_VERIFICACAO
if not os.path.exists(DIRETORIO_SAIDA_XML_SEGUNDA_VERIFICACAO):
    os.makedirs(DIRETORIO_SAIDA_XML_SEGUNDA_VERIFICACAO)
    
# DIRETORIO_SAIDA_XML_SEGUNDA_VERIFICACAO_cProd
if not os.path.exists(DIRETORIO_SAIDA_XML_SEGUNDA_VERIFICACAO_cProd):
    os.makedirs(DIRETORIO_SAIDA_XML_SEGUNDA_VERIFICACAO_cProd)

## Base Excel Classificador

In [3]:
grupo_dict = {grupo: CLASSIFICADOR[CLASSIFICADOR['GRUPO'] == grupo].iloc[:, 1:].to_dict(orient='records')[0] for grupo in CLASSIFICADOR['GRUPO'].unique()}

def valores_grupo(valores):
    try:
        return grupo_dict[valores]
    except:
        return False

## Base Excel Produtos

In [4]:
produtos_BR = PRODUTOS.set_index(list(['COD']))['GRUPO'].to_dict()
produtos_USA = PRODUTOS.set_index(list(['COD-US']))['GRUPO'].to_dict()

# Função para retornar o valor de "GRUPO" com base no valor de "COD"
def encontra_grupo(cod):
    grupoBR, grupoUS = produtos_BR.get(cod), produtos_USA.get(cod)
    if grupoBR is not None:
        return grupoBR
    elif grupoUS is not None:
        return grupoUS
    else:
        return False

# Condição 1.0 – Ignorar os arquivos XML com terminação

In [5]:
def condicao1(arquivos):
    padroes_ignorados = ["*_canc.xml", "*_cteAtz.xml", "*_cteCanc.xml", "*_ciencOp.xml", "*_confirmOp.xml", "*_mdfAtz.xml"]
    return [arquivo for arquivo in arquivos if not any(fnmatch.fnmatch(arquivo, padrao) for padrao in padroes_ignorados)]

# Condição 2.0 – Verificar se o código o <cProd> existe no BD a01 COD

In [6]:
cod_list = np.array(PRODUTOS['COD'])
prod_us = np.array(PRODUTOS['COD-US'])
arr_junto = np.append(cod_list[~np.isnan(cod_list)], prod_us[~np.isnan(prod_us)], axis=0)

grupo_dict = {grupo: CLASSIFICADOR[CLASSIFICADOR['GRUPO'] == grupo].iloc[:, 1:].to_dict(orient='records')[0] for grupo in CLASSIFICADOR['GRUPO'].unique()}

def condicao2(det_elements):
        if all(int(det.find('.//nfe:cProd', namespaces={'nfe': 'http://www.portalfiscal.inf.br/nfe'}).text) in arr_junto for det in det_elements):
            return True

# Condição 3.0 – Verificar se o Elemento <vDesc> existe em cada produto

In [7]:
# Condição 3.0 – Verificar se o Elemento <vDesc> existe em cada produto
def condicao3(det):
    if det.find(".//{http://www.portalfiscal.inf.br/nfe}vDesc") is not None:
        vProd = float(det.find(".//{http://www.portalfiscal.inf.br/nfe}vProd").text)
        vDesc = float(det.find(".//{http://www.portalfiscal.inf.br/nfe}vDesc").text)
        return float(vProd - vDesc)
    else:
        return float(det.find(".//{http://www.portalfiscal.inf.br/nfe}vProd").text)

# Condição 4.1 – Verificar o se o valor do GRUPO > 19

## Condição 4.1.1 – Verificar o se o valor do GRUPO > 19 ICMS

In [8]:
def remove_ICMS_condicao411(det, ns):
    
    
    imposto = det.find('.//nfe:imposto', namespaces=ns)
    if imposto is not None:
        icms = imposto.find('nfe:ICMS', namespaces=ns)
        if icms is not None:
            imposto.remove(icms)
        
        ICMS_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}ICMS')
        icms60_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}ICMS60')
        orig_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}orig')
        cst_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}CST')
        orig_element.text = '0'
        cst_element.text = '60'
        
        icms60_element.append(orig_element)
        icms60_element.append(cst_element)

        ICMS_element.append(icms60_element)
        imposto.append(ICMS_element)

## Condição 4.1.2 – Verificar o se o valor do GRUPO > 19 PIS

In [9]:
def remove_PIS_condicao412(det, vProd, ns):
    
    imposto = det.find('.//nfe:imposto', namespaces=ns)
    if imposto is not None:
        pis = imposto.find('nfe:PIS', namespaces=ns)
        if pis is not None:
            imposto.remove(pis)
        
        PIS_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}PIS')
        pISOutr_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}PISOutr')
        cst_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}CST')
        vBC_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}vBC')
        pPIS_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}pPIS')
        vPIS_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}vPIS')

        cst_element.text = '01'
        vBC_element.text = str(vProd)
        pPIS_element.text = '0.65'
        vPIS_element.text = '{:.2f}'.format(vProd * 0.0065)

        pISOutr_element.append(cst_element)
        pISOutr_element.append(vBC_element)
        pISOutr_element.append(pPIS_element)
        pISOutr_element.append(vPIS_element)

        PIS_element.append(pISOutr_element)
        imposto.append(PIS_element)

## Condição 4.1.3 – Verificar o se o valor do GRUPO > 19 COFINS

In [10]:
def remove_COFINS_condicao413(det, vProd, ns):
            
    imposto = det.find('.//nfe:imposto', namespaces=ns)
    if imposto is not None:
        cofins = imposto.find('nfe:COFINS', namespaces=ns)
        if cofins is not None:
            imposto.remove(cofins)
            
        COFINS_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}COFINS')
        COFINSOutr_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}COFINSOutr')
        cst_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}CST')
        vBC_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}vBC')
        pCOFINS_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}pCOFINS')
        vCOFINS_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}vCOFINS')

        cst_element.text = '01'
        vBC_element.text = '{:.2f}'.format(vProd)
        pCOFINS_element.text = '3.00'
        vCOFINS_element.text = '{:.2f}'.format(vProd * 0.03)

        COFINSOutr_element.append(cst_element)
        COFINSOutr_element.append(vBC_element)
        COFINSOutr_element.append(pCOFINS_element)
        COFINSOutr_element.append(vCOFINS_element)

        COFINS_element.append(COFINSOutr_element)
        imposto.append(COFINS_element)

# Condição 4.2 – Verificar o se o valor do GRUPO > 9 ou GRUPO < 20

## Condição 4.2.1 – Verificar o se o valor do GRUPO > 9 ou GRUPO < 20 ICMS

In [11]:
def remove_ICMS_condicao421(det, ns):
    
    
    imposto = det.find('.//nfe:imposto', namespaces=ns)
    
    if imposto is not None:
        icms = imposto.find('nfe:ICMS', namespaces=ns)
        if icms is not None:
            imposto.remove(icms)
    
        ICMS_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}ICMS')
        icms40_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}ICMS40')
        orig_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}orig')
        cst_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}CST')
        orig_element.text = '0'
        cst_element.text = '41'
        icms40_element.append(orig_element)
        icms40_element.append(cst_element)

        ICMS_element.append(icms40_element)

        imposto.append(ICMS_element)


## Condição 4.2.2 – Verificar o se o valor do GRUPO > 9 ou GRUPO < 20 PIS

In [12]:
def remove_PIS_condicao422(det, ns):
    
    imposto = det.find('.//nfe:imposto', namespaces=ns)
    if imposto is not None:
        pis = imposto.find('nfe:PIS', namespaces=ns)
        if pis is not None:
            imposto.remove(pis)
            
        PIS_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}PIS')
        PISNT_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}PISNT')
        cst_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}CST')
        cst_element.text = '04'
        PISNT_element.append(cst_element)

        PIS_element.append(PISNT_element)
        imposto.append(PIS_element)


## Condição 4.2.3 – Verificar o se o valor do GRUPO > 9 e GRUPO < 20 COFINS

In [13]:
def remove_COFINS_condicao423(det, ns):
    
    imposto = det.find('.//nfe:imposto', namespaces=ns)
    if imposto is not None:
        cofins = imposto.find('nfe:COFINS', namespaces=ns)
        if cofins is not None:
            imposto.remove(cofins)
        
        COFINS_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}COFINS')
        COFINSNT_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}COFINSNT')
        cst_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}CST')
        cst_element.text = '04'
        
        COFINSNT_element.append(cst_element)

        COFINS_element.append(COFINSNT_element)
        imposto.append(COFINS_element)


# Condição 4.3 – Verificar o se o valor do GRUPO < 8

## Condição 4.3.1 – Verificar o se o valor do GRUPO < 8 ICMS

In [14]:
def remove_ICMS_condicao431(det, vProd, ns):
    
    
    imposto = det.find('.//nfe:imposto', namespaces=ns)
    if imposto is not None:
        icms = imposto.find('nfe:ICMS', namespaces=ns)
        if icms is not None:
            imposto.remove(icms)
            
        ICMS_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}ICMS')
        ICMS00_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}ICMS00')
        orig_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}orig')
        CST_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}CST')
        modBC_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}modBC')
        vBC_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}vBC')
        pICMS_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}pICMS')
        vICMS_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}vICMS')

        orig_element.text = '0'
        CST_element.text = '00'
        modBC_element.text = '3'
        vBC_element.text = '{:.2f}'.format(vProd)
        pICMS_element.text = '18.0000'
        vICMS_element.text = '{:.2f}'.format(vProd * 0.18)

        ICMS00_element.append(orig_element)
        ICMS00_element.append(CST_element)
        ICMS00_element.append(modBC_element)
        ICMS00_element.append(vBC_element)
        ICMS00_element.append(pICMS_element)
        ICMS00_element.append(vICMS_element)

        ICMS_element.append(ICMS00_element)
        imposto.append(ICMS_element)

## Condição 4.3.2 – Verificar o se o valor do GRUPO < 8 PIS

In [15]:
def remove_PIS_condicao432(det, vProd, ns):
    imposto = det.find('.//nfe:imposto', namespaces=ns)
    if imposto is not None:
        pis = imposto.find('nfe:PIS', namespaces=ns)
        if pis is not None:
            imposto.remove(pis)
    
        PIS_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}PIS')
        PISAliq_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}PISAliq')
        CST_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}CST')
        vBC_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}vBC')
        pPIS_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}pPIS')
        vPIS_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}vPIS')

        CST_element.text = '01'
        vBC_element.text = '{:.2f}'.format(vProd)
        pPIS_element.text = '0.65'
        vPIS_element.text = '{:.2f}'.format(vProd * 0.0065)

        PISAliq_element.append(CST_element)
        PISAliq_element.append(vBC_element)
        PISAliq_element.append(pPIS_element)
        PISAliq_element.append(vPIS_element)

        PIS_element.append(PISAliq_element)
        imposto.append(PIS_element)

## Condição 4.3.3 – Verificar o se o valor do GRUPO < 8 COFINS

In [16]:
def remove_COFINS_condicao433(det, vProd,ns):
    
    imposto = det.find('.//nfe:imposto', namespaces=ns)
    if imposto is not None:
        cofins = imposto.find('nfe:COFINS', namespaces=ns)
        if cofins is not None:
            imposto.remove(cofins)
        
        COFINS_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}COFINS')
        COFINSAliq_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}COFINSAliq')
        CST_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}CST')
        vBC_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}vBC')
        pCOFINS_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}pCOFINS')
        vCOFINS_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}vCOFINS')

        CST_element.text = '01'
        vBC_element.text = '{:.2f}'.format(vProd)
        pCOFINS_element.text = '3.00'
        vCOFINS_element.text = '{:.2f}'.format(vProd * 0.003)

        COFINSAliq_element.append(CST_element)
        COFINSAliq_element.append(vBC_element)
        COFINSAliq_element.append(pCOFINS_element)
        COFINSAliq_element.append(vCOFINS_element)

        COFINS_element.append(COFINSAliq_element)
        
        imposto.append(COFINS_element)

# Condição 4.4 – Verificar se o valor do GRUPO > 7 e GRUPO < 10

## Condição 4.4.1 – Verificar se o valor do GRUPO > 7 e GRUPO < 10 ICMS

In [17]:
def remove_ICMS_condicao441(det, vProd, ns):
    
    
    imposto = det.find('.//nfe:imposto', namespaces=ns)
    if imposto is not None:
        icms = imposto.find('nfe:ICMS', namespaces=ns)
        if icms is not None:
            imposto.remove(icms)
        
        ICMS_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}ICMS')
        ICMS00_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}ICMS00')
        orig_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}orig')
        CST_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}CST')
        modBC_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}modBC')
        vBC_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}vBC')
        pICMS_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}pICMS')
        vICMS_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}vICMS')

        orig_element.text = '0'
        CST_element.text = '00'
        modBC_element.text = '3'
        vBC_element.text = '{:.2f}'.format(vProd)
        pICMS_element.text = '27.0000'
        vICMS_element.text = '{:.2f}'.format(vProd * 0.27)

        ICMS00_element.append(orig_element)
        ICMS00_element.append(CST_element)
        ICMS00_element.append(modBC_element)
        ICMS00_element.append(vBC_element)
        ICMS00_element.append(pICMS_element)
        ICMS00_element.append(vICMS_element)

        ICMS_element.append(ICMS00_element)
        imposto.append(ICMS_element)

## Condição 4.4.2 – Verificar se o valor do GRUPO = 7 e GRUPO = 10 PIS

In [18]:
def remove_PIS_condicao442(det, vProd, ns):
    imposto = det.find('.//nfe:imposto', namespaces=ns)
    if imposto is not None:
        pis = imposto.find('nfe:PIS', namespaces=ns)
        if pis is not None:
            imposto.remove(pis)
        PIS = ET.Element('{http://www.portalfiscal.inf.br/nfe}PIS')
        PISAliq_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}PISAliq')
        CST_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}CST')
        vBC_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}vBC')
        pPIS_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}pPIS')
        vPIS_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}vPIS')

        CST_element.text = '01'
        vBC_element.text = '{:.2f}'.format(vProd)
        pPIS_element.text = '0.65'
        vPIS_element.text = '{:.2f}'.format(vProd * 0.0065)

        PISAliq_element.append(CST_element)
        PISAliq_element.append(vBC_element)
        PISAliq_element.append(pPIS_element)
        PISAliq_element.append(vPIS_element)

        PIS.append(PISAliq_element)
        imposto.append(PIS)

## Condição 4.4.3 – Verificar se o valor do GRUPO > 7 e GRUPO < 10 CONFINS

In [19]:
def remove_COFINS_condicao443(det, vProd, ns):
    imposto = det.find('.//nfe:imposto', namespaces=ns)
    if imposto is not None:
        cofins = imposto.find('nfe:COFINS', namespaces=ns)
        if cofins is not None:
            imposto.remove(cofins)
    
        COFINS_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}COFINS')
        COFINSAliq_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}COFINSAliq')
        CST_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}CST')
        vBC_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}vBC')
        pCOFINS_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}pCOFINS')
        vCOFINS_element = ET.Element('{http://www.portalfiscal.inf.br/nfe}vCOFINS')

        CST_element.text = '01'
        vBC_element.text = '{:.2f}'.format(vProd)
        pCOFINS_element.text = '3.00'
        vCOFINS_element.text = '{:.2f}'.format(vProd * 0.003)

        COFINSAliq_element.append(CST_element)
        COFINSAliq_element.append(vBC_element)
        COFINSAliq_element.append(pCOFINS_element)
        COFINSAliq_element.append(vCOFINS_element)

        COFINS_element.append(COFINSAliq_element)
        imposto.append(COFINS_element)

# ALTERA ARQUIVO EXCEL CASO NÃO ENCONTRE O cProd NA PLANILHA

In [20]:
def cProd_NAO_encontrado_Excel(diretorio_tabela, det):
    cProd = int(det.find(".//{http://www.portalfiscal.inf.br/nfe}cProd").text)
    xProd = det.find(".//{http://www.portalfiscal.inf.br/nfe}xProd").text
    # Ler o arquivo Excel existente
    tabela_excel = pd.read_excel(diretorio_tabela)

    # Verificar se o número já existe na tabela
    numero_existente = (tabela_excel['COD'].values == cProd).any()

    if numero_existente:
        pass
    else:
        # Adicionar um novo item ao DataFrame
        novo_item = pd.DataFrame({'COD': [cProd],
                                  'COD-US': [''],
                                  'DESCRICAO': [xProd],
                                  'GRUPO': ['']})

        tabela_excel = pd.concat([tabela_excel, novo_item], ignore_index=True)

        # Salvar o DataFrame atualizado no arquivo Excel
        tabela_excel.to_excel(diretorio_tabela, index=False)

# MAIN RODA O PROGRAMA

In [21]:
def principal(arquivos, DIRETORIO_ENTRADA_XML, DIRETORIO_SAIDA_XML, DIRETORIO_SAIDA_cProd_NAO_ENCONTRADO):
    arquivos_filtrados = condicao1(arquivos)
    # Loop para percorrer todos os arquivos XML no diretório de origem
    for nome_arquivo in tqdm(arquivos_filtrados):
        if nome_arquivo.endswith('.xml'):  # Verificar se o arquivo é um arquivo XML
            caminho_arquivo = os.path.join(DIRETORIO_ENTRADA_XML, nome_arquivo)
            xml_file = open(caminho_arquivo, "rb")  # Abrir o arquivo como bytes
            xml_data = xml_file.read()
            xml_file.close()

            ET.register_namespace('', 'http://www.portalfiscal.inf.br/nfe')
            # Parse do XML
            ns = {'nfe': 'http://www.portalfiscal.inf.br/nfe'}
            root = ET.fromstring(xml_data)
            det_elements = root.findall('.//nfe:det', namespaces=ns)
            
            # VERIFICANDO A SEGUNDA CONDIÇÃO
            if condicao2(det_elements):
                    for det in det_elements:
                        valor = valores_grupo(encontra_grupo(int(det.find(".//{http://www.portalfiscal.inf.br/nfe}cProd").text)))
                        if valor:
                            # ALTERANDO O NCM E CFOP DA SEGUNDA CONDIÇÃO
                            det.find(".//{http://www.portalfiscal.inf.br/nfe}NCM").text = str(int(valor['NCM']))
                            # NOVA CONDIÇÃO CFOP
                            if int(det.find(".//{http://www.portalfiscal.inf.br/nfe}CFOP").text) != 5101:
                                det.find(".//{http://www.portalfiscal.inf.br/nfe}CFOP").text = str(int(valor['CFOP-INT']))

                        # VERIFICANDO A QUARTA CONDIÇÃO 4.1
                        if encontra_grupo(int(det.find(".//{http://www.portalfiscal.inf.br/nfe}cProd").text)) > 19:
                            remove_ICMS_condicao411(det, ns)
                            remove_PIS_condicao412(det, condicao3(det), ns)
                            remove_COFINS_condicao413(det, condicao3(det), ns)
                        # VERIFICANDO A QUARTA CONDIÇÃO 4.2    
                        if encontra_grupo(int(det.find(".//{http://www.portalfiscal.inf.br/nfe}cProd").text)) > 9 and encontra_grupo(int(det.find(".//{http://www.portalfiscal.inf.br/nfe}cProd").text)) < 20:
                            remove_ICMS_condicao421(det, ns)
                            remove_PIS_condicao422(det, ns)
                            remove_COFINS_condicao423(det, ns)
                        # VERIFICANDO A QUARTA CONDIÇÃO 4.3  
                        if encontra_grupo(int(det.find(".//{http://www.portalfiscal.inf.br/nfe}cProd").text)) < 8:
                            remove_ICMS_condicao431(det, condicao3(det),ns)
                            remove_PIS_condicao432(det, condicao3(det),ns)
                            remove_COFINS_condicao433(det, condicao3(det),ns)
                        # VERIFICANDO A QUARTA CONDIÇÃO 4.4
                        if encontra_grupo(int(det.find(".//{http://www.portalfiscal.inf.br/nfe}cProd").text)) > 7 and encontra_grupo(int(det.find(".//{http://www.portalfiscal.inf.br/nfe}cProd").text)) < 10:
                            remove_ICMS_condicao441(det, condicao3(det), ns)
                            remove_PIS_condicao442(det, condicao3(det), ns)
                            remove_COFINS_condicao443(det, condicao3(det), ns)

                        # Monta o caminho completo do arquivo de destino
                        tree = ET.ElementTree(root)
                        caminho_arquivo_destino = os.path.join(DIRETORIO_SAIDA_XML, nome_arquivo)
                        tree.write(caminho_arquivo_destino, encoding="utf-8",xml_declaration=True, default_namespace="")
            else:
                for det in det_elements:
                    cProd_NAO_encontrado_Excel(DIRETORIO_TABELA_PRODUTOS,det)
                caminho_arquivo_destino = os.path.join(DIRETORIO_SAIDA_cProd_NAO_ENCONTRADO, nome_arquivo) 
                tree.write(caminho_arquivo_destino, encoding="utf-8", xml_declaration=True, default_namespace="")

# PRIMEIRA VERIFICAÇÃO

In [ ]:
# COMENTAR A SEGUNDA VERIFICAÇÃO QUANDO FOR VERIFICAR A PRIMEIRA 
arquivos = os.listdir(DIRETORIO_ENTRADA_XML)
principal(arquivos, DIRETORIO_ENTRADA_XML, DIRETORIO_SAIDA_XML, DIRETORIO_SAIDA_cProd_NAO_ENCONTRADOS)

# EXECUTAR A SEGUNDA VERIFICAÇÃO APÓS CORRIGGIR OS GRUPOS QUE FALTAM NA TABELA EXCEL DA PRIMEIRA VERIFICAÇÃO

In [ ]:
# COMENTAR A PRIMEIRA VERIFICAÇÃO QUANDO FOR VERIFICAR A SEGUNDA 
#arquivos_segunda_verificacao = os.listdir(DIRETORIO_SAIDA_cProd_NAO_ENCONTRADOS)
#principal(arquivos_segunda_verificacao, DIRETORIO_SAIDA_cProd_NAO_ENCONTRADOS, DIRETORIO_SAIDA_XML_SEGUNDA_VERIFICACAO, DIRETORIO_SAIDA_XML_SEGUNDA_VERIFICACAO_cProd)